# This notebook processes CAFE v3 ocean daily data for building climatologies. Only the last 100 years are used.
Currently only runs on Raijin, as control run data not yet transferred to Canberra

In [1]:
# Import packages -----
import pandas as pd
import xarray as xr
import numpy as np
from ipywidgets import FloatProgress
from dateutil.relativedelta import relativedelta

#### Initialise

In [2]:
# Standard naming -----
fields = pd.DataFrame( \
        {'name_CAFE': ['sst', 'patm_t', 'eta_t', 'sss', 'u_surf', 'v_surf', 'mld'],
         'name_std' : ['sst', 'patm',   'eta',   'sss', 'u_s',    'v_s',    'mld']}
                     )
name_dict = fields.set_index('name_CAFE').to_dict()['name_std']

fields

,name_CAFE,name_std
0,sst,sst
1,patm_t,patm
2,eta_t,eta
3,sss,sss
4,u_surf,u_s
5,v_surf,v_s
6,mld,mld


#### Only use last 100 years

In [3]:
# Loop over all paths -----
base = '/g/data1/v14/coupled_model/v3/OUTPUT/'
years = range(400,500)

paths = []
for year in years:
    path = base + 'ocean_daily_0' + str(year) + '_01_01.nc'
    paths.append(path)

ds = xr.open_mfdataset(paths, autoclose=True) \
       .drop(['average_T1','average_T2','average_DT','time_bounds',
              'area_t','area_u','geolat_c','geolat_t','ht']) \
       .rename(name_dict)
        
if 'xu_ocean' in ds.dims:
    ds = ds.rename({'xu_ocean':'lon_u','yu_ocean':'lat_u'})
if 'xt_ocean' in ds.dims:
    ds = ds.rename({'xt_ocean':'lon_t','yt_ocean':'lat_t'})

FileNotFoundError: [Errno 2] No such file or directory: b'/g/data1/v14/coupled_model/v3/OUTPUT/ocean_daily_0400_01_01.nc'

In [8]:
# Use year 2016 as time -----
path = '/g/data1/v14/forecast/v1/yr2016/mn1/OUTPUT.1/ocean_daily*.nc'
dataset = xr.open_mfdataset(path, autoclose=True)
time_use = xr.concat([dataset.time[:59], dataset.time[60:366]],dim='time')
time_ly = dataset.time[59]

In [9]:
# Make month_day array of month-day -----
m = [str(ds.time.values[i].timetuple()[1]).zfill(2) + '-' for i in range(len(ds.time))]
d = [str(ds.time.values[i].timetuple()[2]).zfill(2) for i in range(len(ds.time))]
md = np.core.defchararray.add(m, d)

# Replace time array with month_day array and groupby -----
ds['time'] = md
clim = ds.groupby('time').mean(dim='time',keep_attrs=True)
clim['time'] = time_use

In [11]:
# Replicate Feb 28th as Feb 29th to deal with leap years -----
clim_ly = clim.copy().sel(time='2016-02-28')
clim_ly['time'] = np.array([time_ly.values])
clim = xr.auto_combine([clim,clim_ly]).sortby('time')

In [ ]:
# Save the climatology -----
save_fldr = '/g/data1/v14/squ027/tmp/'
clim.to_netcdf(save_fldr + 'cafe.c3.ocean.400_499.clim.nc', mode = 'w',
               encoding = {'time':{'dtype':'float','calendar':'JULIAN',
                           'units':'days since 0001-01-01 00:00:00'}})